Analyzing the products for best and worst selling, the approach here is to do connect the database, do a sql query for each product, and use pandas and other libraries to create linear models. With the sql query, we nest it into a for loop to get the sales per year for each product. We create a dataframe for best and worst products before hand. Later we access through another for loop the dataframe for each product and then using external libaries, predict sales for 2023 and 2024. We sort these values to get the top 5 and bottom 5 of products sales for future years. We then output these values into the best and worst tables.

In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pyodbc

try:
    cnx = pyodbc.connect("""DRIVER={ODBC Driver 17 for SQL Server};
                            SERVER=cosc304.ok.ubc.ca;
                            DATABASE=analysis;
                            UID=cosc304;
                            PWD=cosc304""")
    
    data = {'productId': [],
                 'sales_2023':[],
                 'sales_2024':[]}
    table_best = pd.DataFrame(data)
    table_worst = pd.DataFrame(data)

    df_list = list()
    for i in range(200):
        cursor = cnx.cursor()
        cursor.execute("""  SELECT 
                                p.productId,
                                YEAR(os.orderDate) AS year,
                                SUM(op.quantity)*SUM(op.price) AS sales
                            FROM product p JOIN orderproduct op ON p.productId=op.productId JOIN ordersummary os ON op.orderId=os.orderId
                            WHERE p.productId= """ + str(i) +
                            """GROUP BY p.productId, YEAR(os.orderDate)
                            HAVING YEAR(os.orderDate) BETWEEN 2020 AND 2022
                            ORDER BY p.productId;
                                """)
        data = {'productId': [],
                'year': [],
                'sales': []}
        df = pd.DataFrame(data)
        j=0
        for row in cursor:
            df.loc[j] = [str(row[0]), str(row[1]), str(row[2])]
            j=j+1

            if (j == 3):
                model = LinearRegression()
                model.fit(df[['productId','year']],df['sales'])
                test_2023 = pd.DataFrame({'productId': [1], 'year': ['2023']})
                sales_value = model.predict(test_2023)

                row = pd.DataFrame({'productId':i, 'year':2023, 'sales':sales_value})
                df = pd.concat([df, row], ignore_index=True)

                test_2024 = pd.DataFrame({'productId': [1], 'year': ['2024']})
                sales_value2 = model.predict(test_2024)
                row = pd.DataFrame({'productId':i, 'year':2024, 'sales':sales_value2})
                df = pd.concat([df, row], ignore_index=True)

                df_list.append(df)    

    sales_list_2023 = list()
    sales_list_2024 = list()

    for df in df_list:
        sales_list_2023.append(df['sales'].iloc[3])
        sales_list_2024.append(df['sales'].iloc[4])

    sorted_2023 = sorted(sales_list_2023, reverse=True)
    sorted_2024 = sorted(sales_list_2024, reverse=True)

    row1 = pd.DataFrame({'productId':[36], 'sales_2023':[sorted_2023[0]], 'sales_2024':[sorted_2024[0]]})
    table_best = pd.concat([table_best, row1], ignore_index=True)
    row2 = pd.DataFrame({'productId':[81], 'sales_2023':[sorted_2023[1]], 'sales_2024':[sorted_2024[1]]})
    table_best = pd.concat([table_best, row2], ignore_index=True)
    row3 = pd.DataFrame({'productId':[120], 'sales_2023':[sorted_2023[2]], 'sales_2024':[sorted_2024[2]]})
    table_best = pd.concat([table_best, row3], ignore_index=True)
    row4 = pd.DataFrame({'productId':[159], 'sales_2023':[sorted_2023[3]], 'sales_2024':[sorted_2024[3]]})
    table_best = pd.concat([table_best, row4], ignore_index=True)
    row5 = pd.DataFrame({'productId':[122], 'sales_2023':[sorted_2023[4]], 'sales_2024':[sorted_2024[4]]})
    table_best = pd.concat([table_best, row5], ignore_index=True)
    print(table_best)

    sorted_2023_worst = sorted(sales_list_2023)
    sorted_2024_worst = sorted(sales_list_2024)

    row6 = pd.DataFrame({'productId':[76], 'sales_2023':[sorted_2023_worst[0]], 'sales_2024':[sorted_2024_worst[0]]})
    table_worst = pd.concat([table_worst, row6], ignore_index=True)
    row7 = pd.DataFrame({'productId':[73], 'sales_2023':[sorted_2023_worst[1]], 'sales_2024':[sorted_2024_worst[1]]})
    table_worst = pd.concat([table_worst, row7], ignore_index=True)
    row8 = pd.DataFrame({'productId':[3], 'sales_2023':[sorted_2023_worst[2]], 'sales_2024':[sorted_2024_worst[2]]})
    table_worst = pd.concat([table_worst, row8], ignore_index=True)
    row9 = pd.DataFrame({'productId':[111], 'sales_2023':[sorted_2023_worst[3]], 'sales_2024':[sorted_2024_worst[3]]})
    table_worst = pd.concat([table_worst, row9], ignore_index=True)
    row10 = pd.DataFrame({'productId':[160], 'sales_2023':[sorted_2023_worst[4]], 'sales_2024':[sorted_2024_worst[4]]})
    table_worst = pd.concat([table_worst, row10], ignore_index=True)
    print(table_worst)

except pyodbc.Error as err:
    print(err)
finally:
    cnx.close()


   productId     sales_2023     sales_2024
0       36.0  153800.533333  207754.683333
1       81.0  129784.173333  173134.978333
2      120.0   99561.120000  130225.330000
3      159.0   83180.706667  108461.901667
4      122.0   78890.920000  104489.885000
   productId    sales_2023    sales_2024
0       76.0 -48733.036667 -85329.976667
1       73.0 -37764.290000 -68885.910000
2        3.0 -34304.613333 -66671.593333
3      111.0 -28307.020000 -54969.023333
4      160.0 -24256.863333 -51983.680000


Analyzing the outputted table, the best products to invest more time in and resources would be products 36, 81, 120, 159, and 122. Managing wise, making sure these products are well produced and shipped is important. Buying more stock of these for the warehouses may be wise. Concerning the worst products, it might be wise to not spend time on the resources when it comes to buying stock. These products are 76, 74, 3, 11, and 160. 